# Import Relevant Data

In [1]:
import pandas as pd
import ast
import numpy as np
import math
import time
from pandas.core.frame import DataFrame
import matplotlib.pyplot as plt 
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

In [2]:
df_cryp=pd.read_csv('https://raw.githubusercontent.com/SunYutongAmber/SignatureWork/main/Data/gas_rarity_usd_merged.csv')
df_cryp

,Unnamed: 0,Unnamed: 0.1,date_x,from,to,eth_price,punk_id,type,gender,skin_tone,...,skin_tone_color,img_url,Rarity_Score_1_rarest,Rarity_Score_2_Average,Rarity_Score_3_multiple,timestamp,date_y,gas_eth_price,date,eth_usd_price
0,2458,2458,2018-01-18,2794,15,0.15,5289,Human,Male,Dark,...,#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...,12.040656,9.048897,72.931919,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01
1,2459,2459,2018-01-18,809,920,0.09,8994,Human,Female,Dark,...,#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...,74.786090,26.889225,486.899335,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01
2,2462,2462,2018-01-18,1303,2450,0.09,4679,Human,Male,Albino,...,#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...,7.460154,6.064456,34.829651,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01
3,319,319,2018-01-18,1303,2030,0.25,9863,Human,Male,Dark,...,#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...,1.478706,1.478706,1.478706,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01
4,2473,2473,2018-01-19,1303,5126,0.14,1575,Human,Male,Medium,...,#DB9065,https://www.larvalabs.com/cryptopunks/cryptopu...,7.585070,5.654169,166.565775,1.516320e+09,2018-1-19,0.052589,2018-1-19,979.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16775,12450,12450,2022-07-26,980,1939,94.00,9099,Human,Male,Light,...,#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...,5.427700,3.061263,0.000000,1.658794e+09,2022-7-26,0.001820,2022-7-26,1450.91
16776,17440,17440,2022-07-26,5318,4950,69.00,4430,Human,Female,Albino,...,#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...,4.715707,4.285346,18.178976,1.658794e+09,2022-7-26,0.001820,2022-7-26,1450.91
16777,17613,17613,2022-07-26,3002,1044,94.90,5600,Human,Male,Albino,...,#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...,18.331532,8.951758,329.930659,1.658794e+09,2022-7-26,0.001820,2022-7-26,1450.91
16778,13214,13214,2022-07-27,3096,1028,69.69,6221,Human,Female,Dark,...,#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...,19.315489,7.224599,0.000000,1.658880e+09,2022-7-27,0.001720,2022-7-27,1635.98


# Volume of Active Market Wallet & Daily Change Rate

In [3]:
df_group=df_cryp.groupby('date_x').count()
active_wallet=np.array(df_group['Unnamed: 0'])*2
df_wallet=DataFrame({"active_wallet": active_wallet,
                     "date":df_group.index})
df_wallet

,active_wallet,date
0,8,2018-01-18
1,12,2018-01-19
2,38,2018-01-20
3,4,2018-01-22
4,12,2018-01-23
...,...,...
1212,4,2022-07-22
1213,4,2022-07-23
1214,12,2022-07-25
1215,6,2022-07-26


In [4]:
df_wallet['active_wallet_shift']=df_wallet['active_wallet'].shift(1)
df_wallet['active_wallet_shift'][0]=3
df_wallet

<ipython-input-4-51ec93477ae8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wallet['active_wallet_shift'][0]=3


,active_wallet,date,active_wallet_shift
0,8,2018-01-18,3.0
1,12,2018-01-19,8.0
2,38,2018-01-20,12.0
3,4,2018-01-22,38.0
4,12,2018-01-23,4.0
...,...,...,...
1212,4,2022-07-22,4.0
1213,4,2022-07-23,4.0
1214,12,2022-07-25,4.0
1215,6,2022-07-26,12.0


In [5]:
wallet_change=(np.array(df_wallet['active_wallet'])-np.array(df_wallet['active_wallet_shift']))/np.array(df_wallet['active_wallet_shift'])
df_wallet['wallet_change']=wallet_change
df_wallet['wallet_change']

0       1.666667
1       0.500000
2       2.166667
3      -0.894737
4       2.000000
          ...   
1212    0.000000
1213    0.000000
1214    2.000000
1215   -0.500000
1216   -0.333333
Name: wallet_change, Length: 1217, dtype: float64

In [6]:
df_cryp=df_cryp.merge(df_wallet, left_on='date_x', right_on='date')

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'date_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


In [7]:
df_cryp.to_csv('wallet_rarity_usd_gas_merged.csv')
df_cryp

,Unnamed: 0,Unnamed: 0.1,date_x,from,to,eth_price,punk_id,type,gender,skin_tone,...,Rarity_Score_3_multiple,timestamp,date_y,gas_eth_price,date_x,eth_usd_price,active_wallet,date_y,active_wallet_shift,wallet_change
0,2458,2458,2018-01-18,2794,15,0.15,5289,Human,Male,Dark,...,72.931919,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01,8,2018-01-18,3.0,1.666667
1,2459,2459,2018-01-18,809,920,0.09,8994,Human,Female,Dark,...,486.899335,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01,8,2018-01-18,3.0,1.666667
2,2462,2462,2018-01-18,1303,2450,0.09,4679,Human,Male,Albino,...,34.829651,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01,8,2018-01-18,3.0,1.666667
3,319,319,2018-01-18,1303,2030,0.25,9863,Human,Male,Dark,...,1.478706,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01,8,2018-01-18,3.0,1.666667
4,2473,2473,2018-01-19,1303,5126,0.14,1575,Human,Male,Medium,...,166.565775,1.516320e+09,2018-1-19,0.052589,2018-1-19,979.57,12,2018-01-19,8.0,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16775,12450,12450,2022-07-26,980,1939,94.00,9099,Human,Male,Light,...,0.000000,1.658794e+09,2022-7-26,0.001820,2022-7-26,1450.91,6,2022-07-26,12.0,-0.500000
16776,17440,17440,2022-07-26,5318,4950,69.00,4430,Human,Female,Albino,...,18.178976,1.658794e+09,2022-7-26,0.001820,2022-7-26,1450.91,6,2022-07-26,12.0,-0.500000
16777,17613,17613,2022-07-26,3002,1044,94.90,5600,Human,Male,Albino,...,329.930659,1.658794e+09,2022-7-26,0.001820,2022-7-26,1450.91,6,2022-07-26,12.0,-0.500000
16778,13214,13214,2022-07-27,3096,1028,69.69,6221,Human,Female,Dark,...,0.000000,1.658880e+09,2022-7-27,0.001720,2022-7-27,1635.98,4,2022-07-27,6.0,-0.333333


# Volume of Cryptopunks Sales Daily Change Rate

In [8]:
df_cryp

,Unnamed: 0,Unnamed: 0.1,date_x,from,to,eth_price,punk_id,type,gender,skin_tone,...,Rarity_Score_3_multiple,timestamp,date_y,gas_eth_price,date_x,eth_usd_price,active_wallet,date_y,active_wallet_shift,wallet_change
0,2458,2458,2018-01-18,2794,15,0.15,5289,Human,Male,Dark,...,72.931919,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01,8,2018-01-18,3.0,1.666667
1,2459,2459,2018-01-18,809,920,0.09,8994,Human,Female,Dark,...,486.899335,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01,8,2018-01-18,3.0,1.666667
2,2462,2462,2018-01-18,1303,2450,0.09,4679,Human,Male,Albino,...,34.829651,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01,8,2018-01-18,3.0,1.666667
3,319,319,2018-01-18,1303,2030,0.25,9863,Human,Male,Dark,...,1.478706,1.516234e+09,2018-1-18,0.056103,2018-1-18,991.01,8,2018-01-18,3.0,1.666667
4,2473,2473,2018-01-19,1303,5126,0.14,1575,Human,Male,Medium,...,166.565775,1.516320e+09,2018-1-19,0.052589,2018-1-19,979.57,12,2018-01-19,8.0,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16775,12450,12450,2022-07-26,980,1939,94.00,9099,Human,Male,Light,...,0.000000,1.658794e+09,2022-7-26,0.001820,2022-7-26,1450.91,6,2022-07-26,12.0,-0.500000
16776,17440,17440,2022-07-26,5318,4950,69.00,4430,Human,Female,Albino,...,18.178976,1.658794e+09,2022-7-26,0.001820,2022-7-26,1450.91,6,2022-07-26,12.0,-0.500000
16777,17613,17613,2022-07-26,3002,1044,94.90,5600,Human,Male,Albino,...,329.930659,1.658794e+09,2022-7-26,0.001820,2022-7-26,1450.91,6,2022-07-26,12.0,-0.500000
16778,13214,13214,2022-07-27,3096,1028,69.69,6221,Human,Female,Dark,...,0.000000,1.658880e+09,2022-7-27,0.001720,2022-7-27,1635.98,4,2022-07-27,6.0,-0.333333


In [9]:
sales=df_cryp.groupby('timestamp')['eth_price'].sum()
sales

timestamp
1.516234e+09      0.58
1.516320e+09      0.79
1.516406e+09      2.46
1.516579e+09      0.80
1.516666e+09      1.24
                 ...  
1.658448e+09    177.00
1.658534e+09    198.00
1.658707e+09    410.40
1.658794e+09    257.90
1.658880e+09    174.64
Name: eth_price, Length: 1217, dtype: float64

In [10]:
df_sales=DataFrame({'daily_sales':sales,
                    'date_z':df_group.index})
df_sales

,daily_sales,date_z
timestamp,,
1.516234e+09,0.58,2018-01-18
1.516320e+09,0.79,2018-01-19
1.516406e+09,2.46,2018-01-20
1.516579e+09,0.80,2018-01-22
1.516666e+09,1.24,2018-01-23
...,...,...
1.658448e+09,177.00,2022-07-22
1.658534e+09,198.00,2022-07-23
1.658707e+09,410.40,2022-07-25


In [11]:
df_sales['sales_shift']=df_sales['daily_sales'].shift(1)
df_sales['sales_shift'][0]=0.42
change_sales=(np.array(df_sales['daily_sales'])-np.array(df_sales['sales_shift']))/np.array(df_sales['sales_shift'])
change_sales

<ipython-input-11-c0bd02db3cfe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sales['sales_shift'][0]=0.42
<ipython-input-11-c0bd02db3cfe>:3: RuntimeWarning: divide by zero encountered in true_divide
  change_sales=(np.array(df_sales['daily_sales'])-np.array(df_sales['sales_shift']))/np.array(df_sales['sales_shift'])


array([ 0.38095238,  0.36206897,  2.11392405, ...,  1.07272727,
       -0.37158869, -0.32283831])

In [12]:
df_sales['daily_sales_change']=change_sales
df_sales

,daily_sales,date_z,sales_shift,daily_sales_change
timestamp,,,,
1.516234e+09,0.58,2018-01-18,0.42,0.380952
1.516320e+09,0.79,2018-01-19,0.58,0.362069
1.516406e+09,2.46,2018-01-20,0.79,2.113924
1.516579e+09,0.80,2018-01-22,2.46,-0.674797
1.516666e+09,1.24,2018-01-23,0.80,0.550000
...,...,...,...,...
1.658448e+09,177.00,2022-07-22,168.68,0.049324
1.658534e+09,198.00,2022-07-23,177.00,0.118644
1.658707e+09,410.40,2022-07-25,198.00,1.072727


In [13]:
df_cryp=df_cryp.merge(df_sales,left_on='timestamp',right_on='timestamp')

In [14]:
df_cryp.to_csv('sales_wallet_rarity_gas_usd.csv')

In [15]:
df_cryp

,Unnamed: 0,Unnamed: 0.1,date_x,from,to,eth_price,punk_id,type,gender,skin_tone,...,date_x,eth_usd_price,active_wallet,date_y,active_wallet_shift,wallet_change,daily_sales,date_z,sales_shift,daily_sales_change
0,2458,2458,2018-01-18,2794,15,0.15,5289,Human,Male,Dark,...,2018-1-18,991.01,8,2018-01-18,3.0,1.666667,0.58,2018-01-18,0.42,0.380952
1,2459,2459,2018-01-18,809,920,0.09,8994,Human,Female,Dark,...,2018-1-18,991.01,8,2018-01-18,3.0,1.666667,0.58,2018-01-18,0.42,0.380952
2,2462,2462,2018-01-18,1303,2450,0.09,4679,Human,Male,Albino,...,2018-1-18,991.01,8,2018-01-18,3.0,1.666667,0.58,2018-01-18,0.42,0.380952
3,319,319,2018-01-18,1303,2030,0.25,9863,Human,Male,Dark,...,2018-1-18,991.01,8,2018-01-18,3.0,1.666667,0.58,2018-01-18,0.42,0.380952
4,2473,2473,2018-01-19,1303,5126,0.14,1575,Human,Male,Medium,...,2018-1-19,979.57,12,2018-01-19,8.0,0.500000,0.79,2018-01-19,0.58,0.362069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16775,12450,12450,2022-07-26,980,1939,94.00,9099,Human,Male,Light,...,2022-7-26,1450.91,6,2022-07-26,12.0,-0.500000,257.90,2022-07-26,410.40,-0.371589
16776,17440,17440,2022-07-26,5318,4950,69.00,4430,Human,Female,Albino,...,2022-7-26,1450.91,6,2022-07-26,12.0,-0.500000,257.90,2022-07-26,410.40,-0.371589
16777,17613,17613,2022-07-26,3002,1044,94.90,5600,Human,Male,Albino,...,2022-7-26,1450.91,6,2022-07-26,12.0,-0.500000,257.90,2022-07-26,410.40,-0.371589
16778,13214,13214,2022-07-27,3096,1028,69.69,6221,Human,Female,Dark,...,2022-7-27,1635.98,4,2022-07-27,6.0,-0.333333,174.64,2022-07-27,257.90,-0.322838


# Cryptopunk Price(USD)

In [16]:
df_cryp['Price']=np.array(df_cryp['eth_price'])*np.array(df_cryp['eth_usd_price'])
df_cryp

,Unnamed: 0,Unnamed: 0.1,date_x,from,to,eth_price,punk_id,type,gender,skin_tone,...,eth_usd_price,active_wallet,date_y,active_wallet_shift,wallet_change,daily_sales,date_z,sales_shift,daily_sales_change,Price
0,2458,2458,2018-01-18,2794,15,0.15,5289,Human,Male,Dark,...,991.01,8,2018-01-18,3.0,1.666667,0.58,2018-01-18,0.42,0.380952,148.6515
1,2459,2459,2018-01-18,809,920,0.09,8994,Human,Female,Dark,...,991.01,8,2018-01-18,3.0,1.666667,0.58,2018-01-18,0.42,0.380952,89.1909
2,2462,2462,2018-01-18,1303,2450,0.09,4679,Human,Male,Albino,...,991.01,8,2018-01-18,3.0,1.666667,0.58,2018-01-18,0.42,0.380952,89.1909
3,319,319,2018-01-18,1303,2030,0.25,9863,Human,Male,Dark,...,991.01,8,2018-01-18,3.0,1.666667,0.58,2018-01-18,0.42,0.380952,247.7525
4,2473,2473,2018-01-19,1303,5126,0.14,1575,Human,Male,Medium,...,979.57,12,2018-01-19,8.0,0.500000,0.79,2018-01-19,0.58,0.362069,137.1398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16775,12450,12450,2022-07-26,980,1939,94.00,9099,Human,Male,Light,...,1450.91,6,2022-07-26,12.0,-0.500000,257.90,2022-07-26,410.40,-0.371589,136385.5400
16776,17440,17440,2022-07-26,5318,4950,69.00,4430,Human,Female,Albino,...,1450.91,6,2022-07-26,12.0,-0.500000,257.90,2022-07-26,410.40,-0.371589,100112.7900
16777,17613,17613,2022-07-26,3002,1044,94.90,5600,Human,Male,Albino,...,1450.91,6,2022-07-26,12.0,-0.500000,257.90,2022-07-26,410.40,-0.371589,137691.3590
16778,13214,13214,2022-07-27,3096,1028,69.69,6221,Human,Female,Dark,...,1635.98,4,2022-07-27,6.0,-0.333333,174.64,2022-07-27,257.90,-0.322838,114011.4462


# ROI

## roi1

In [17]:
number = df_cryp.groupby(['punk_id']).count()
max(number['type'])

15

In [18]:
roi = df_cryp.groupby(['punk_id']).agg({'Price':['min','max']})
#roi.rename(columns={'data1':'data1_min'},inplace=True)#修改列名
roi=roi['Price']
roi

,min,max
punk_id,,
1,5129.880000,5129.8800
14,232.800000,232.8000
53,27461.400000,431963.2695
89,7049.473500,7049.4735
118,36928.102500,66122.2120
...,...,...
9980,80928.530400,87877.5375
9982,404981.100000,483357.4900
9990,0.029114,225545.1070


In [19]:
roi['ROI'] = (roi['max']-roi['min'])/roi['min']
roi

,min,max,ROI
punk_id,,,
1,5129.880000,5129.8800,0.000000e+00
14,232.800000,232.8000,0.000000e+00
53,27461.400000,431963.2695,1.472983e+01
89,7049.473500,7049.4735,0.000000e+00
118,36928.102500,66122.2120,7.905662e-01
...,...,...,...
9980,80928.530400,87877.5375,8.586597e-02
9982,404981.100000,483357.4900,1.935310e-01
9990,0.029114,225545.1070,7.746961e+06


## roi2

In [20]:
roi2 = df_cryp.groupby(['punk_id'])['timestamp','punk_id','Price']
roi2.get_group(197)

<ipython-input-20-5cb3efb4e904>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  roi2 = df_cryp.groupby(['punk_id'])['timestamp','punk_id','Price']


,timestamp,punk_id,Price
14026,1.638490e+09,197,295244.60
14339,1.640822e+09,197,241112.30
14343,1.640822e+09,197,237402.88
14545,1.642810e+09,197,0.00
14563,1.642810e+09,197,0.00


In [21]:
df_cryp.loc[1,'timestamp']

1516233600.0

In [24]:
list_roi=[]
rq=[] #date
for i in roi.index:
  if len(list(roi2.get_group(i).index)) >1:
    for j in range(len(list(roi2.get_group(i).index)[0:-1])):
     
      a=(roi2.get_group(i).iloc[j+1,2]-roi2.get_group(i).iloc[j,2])/roi2.get_group(i).iloc[j,2]
      list_roi.append(a) 
      b=df_cryp.loc[j,'timestamp']
      rq.append(b)

list_roi

<ipython-input-24-3b6c0ea0f800>:7: RuntimeWarning: invalid value encountered in double_scalars
  a=(roi2.get_group(i).iloc[j+1,2]-roi2.get_group(i).iloc[j,2])/roi2.get_group(i).iloc[j,2]
<ipython-input-24-3b6c0ea0f800>:7: RuntimeWarning: divide by zero encountered in double_scalars
  a=(roi2.get_group(i).iloc[j+1,2]-roi2.get_group(i).iloc[j,2])/roi2.get_group(i).iloc[j,2]


[0.5663265529069897,
 9.04249988143263,
 0.7905661954875693,
 0.40928969286678923,
 0.15086771950939581,
 0.05172413793103442,
 3.3118160493741926,
 0.38682406568196126,
 1.8534539879090142,
 0.1255764769126302,
 0.9659225474576087,
 0.23432695568093984,
 0.11167395438016321,
 0.0693617572645375,
 0.9918685590020917,
 3.1308585501068396,
 0.16194805194805195,
 61.12215450409951,
 0.48064783753562046,
 3.20780529694719,
 -0.18334729915466688,
 -0.015384615384615436,
 -1.0,
 nan,
 6.806518552845658,
 -0.25404451075469364,
 0.24902975251792753,
 -0.5160996882729567,
 8.101871228951962,
 0.07382166725290178,
 0.12989118466670083,
 0.3366015571488037,
 1.2872286999339204,
 -0.010264464916849644,
 1.6359998402659988,
 0.7923502673050624,
 0.2154625967781259,
 0.07969721726661887,
 0.36605127240582425,
 0.45613584776763416,
 0.5125005566888552,
 4.432368728191631,
 0.7193876090194389,
 0.0,
 -0.8063921395537411,
 -0.36889368762668184,
 -0.10607751519430993,
 3.284402107905663,
 25.94226093829

In [25]:
ROI=DataFrame({'Time':rq,
               'ROI':list_roi})
ROI

,Time,ROI
0,1.516234e+09,5.663266e-01
1,1.516234e+09,9.042500e+00
2,1.516234e+09,7.905662e-01
3,1.516234e+09,4.092897e-01
4,1.516234e+09,1.508677e-01
...,...,...
10866,1.516234e+09,-1.621499e-01
10867,1.516234e+09,-9.999993e-01
10868,1.516234e+09,7.746961e+06
10869,1.516234e+09,4.802105e+00


In [26]:
df_cryp=df_cryp.merge(ROI,left_on='timestamp',right_on='Time')

In [31]:
df_cryp=df_cryp[df_cryp['ROI']<100000]

In [36]:
df_cryp.to_csv('roi_sales_wallet_rarity_gas_usd.csv')

# Explaination

In [ ]:
#df_cryp=pd.read_csv('https://raw.githubusercontent.com/HCI-Blockchain/ICWSM-2023/main/data/hedonic_data/roi_sales_wallet_rarity_gas_usd.csv')
df_cryp.columns

## Correlation

# Stationay Test

## Augmented Dickey Fuller test (ADF Test)：

零假设为序列有单位根，是非*稳的，P-Value如果小于显著级别(0.05)，则可以拒绝零假设。

In [28]:
from statsmodels.tsa.stattools import adfuller, kpss
df = df_cryp

# ADF Test
result = adfuller(df.Price.values, autolag='AIC')
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
for key, value in result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')

ADF Statistic: -2.322598172667917
p-value: 0.16478315340256638
Critial Values:
   1%, -3.4304898737875784
Critial Values:
   5%, -2.861601821246897
Critial Values:
   10%, -2.5668029054215773


In [33]:
# ADF Test
df = df_cryp
result = adfuller(df.ROI.values, autolag='AIC')
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
for key, value in result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')

ADF Statistic: -31.245106689534538
p-value: 0.0
Critial Values:
   1%, -3.430495617319286
Critial Values:
   5%, -2.8616043597037923
Critial Values:
   10%, -2.5668042565697466


## Kwiatkowski-Phillips-Schmidt-Shin – KPSS test (trend stationary)：

与ADF正好相反，零假设为序列是*稳的。另外，在python中，可以通过指定regression='ct'参数来让kps把“确定性趋势(deterministic trend)”的序列认为是*稳的。

In [34]:
# KPSS Test
result = kpss(df.Price.values, regression='c')
print('\nKPSS Statistic: %f' % result[0])
print('p-value: %f' % result[1])
for key, value in result[3].items():
    print('Critial Values:')
    print(f'   {key}, {value}')


KPSS Statistic: 16.267353
p-value: 0.010000
Critial Values:
   10%, 0.347
Critial Values:
   5%, 0.463
Critial Values:
   2.5%, 0.574
Critial Values:
   1%, 0.739


/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/stattools.py:1875: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/stattools.py:1906: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warnings.warn(


In [35]:
# KPSS Test
result = kpss(df.ROI.values, regression='c')
print('\nKPSS Statistic: %f' % result[0])
print('p-value: %f' % result[1])
for key, value in result[3].items():
    print('Critial Values:')
    print(f'   {key}, {value}')


KPSS Statistic: 7.052417
p-value: 0.010000
Critial Values:
   10%, 0.347
Critial Values:
   5%, 0.463
Critial Values:
   2.5%, 0.574
Critial Values:
   1%, 0.739


/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/stattools.py:1906: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warnings.warn(
